In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from numpy import uint8
class mouseParam:
    def __init__(self, input_img_name):
        self.mouseEvent = {"w":None, "h":None, "event":None, "flags":None}
        cv2.setMouseCallback(input_img_name, self.__CallBackFunc, None)
    
    def __CallBackFunc(self, eventType, w, h, flags, userdata):
        
        self.mouseEvent["w"] = w
        self.mouseEvent["h"] = h
        self.mouseEvent["event"] = eventType    
        self.mouseEvent["flags"] = flags    

    #マウス入力用のパラメータを返すための関数
    def getData(self):
        return self.mouseEvent
    
    #マウスイベントを返す関数
    def getEvent(self):
        return self.mouseEvent["event"]                

    #マウスフラグを返す関数
    def getFlags(self):
        return self.mouseEvent["flags"]                

    #マウスが座標を返す関数
    def getPos(self):
        return (self.mouseEvent["h"], self.mouseEvent["w"])
        

if __name__ == "__main__":
    plt.ion()
    image = cv2.imread('images/mt_fuji.jpg', cv2.IMREAD_GRAYSCALE)
    window_name = "input window"
    
    h,w = image.shape
    F1= np.zeros(shape=(h,w))
    F2= np.zeros(shape=(h,w))
    F3= np.zeros(shape=(h,w))
    
    ff = np.fft.fft2(image)
    fshift = np.fft.fftshift(ff)
    magnitude = 20*np.log(np.abs(fshift))
    magmin = np.min(magnitude)
    magmax = np.max(magnitude)
    mag2 = (magnitude - magmin) * 255/(magmax - magmin)
    newmag = np.uint8(mag2)
    cv2.imshow("fft",newmag)
    
    cv2.imshow("mouse",F2)
    mouseData = mouseParam("mouse")
    
    F1_4= np.zeros(shape=(h,w))
    cv2.imshow(window_name, image)
    
    draw = False
    while True:
        k = cv2.waitKey(1)
        if k == 27:
            break
        
        cv2.imshow(window_name, image)
        
        if mouseData.getEvent() == cv2.EVENT_LBUTTONDOWN:
            draw = True
            
        if mouseData.getEvent() == cv2.EVENT_LBUTTONUP:
            draw = False
        
        
        if mouseData.getEvent() == cv2.EVENT_MOUSEMOVE and draw:
            h1,w1 = mouseData.getPos()
            F2[h1:h1+2,w1:w1+2]=255
            F1[h1:h1+2,w1:w1+2]=1.0
            F3[:,:]=0
            F3[mouseData.getPos()]=1
            
            F1_2 = fshift*F1
            F1_3 = np.fft.fftshift(F1_2)
            F1_iff= np.fft.ifft2(F1_3)
            F1_abs =np.abs(F1_iff) 
            F1_max = np.max(F1_abs)
            F1_min = np.min(F1_abs)
            F1_4 = (F1_abs - F1_min )* 255/(F1_max - F1_min)
            
            F3 = F3*ff
            F3 = np.fft.fftshift(F3)
            F3 = np.fft.ifft2(F3)
            F3 = np.abs(F3)
            F3_max = F3.max()
            F3_min = F3.min()
            F3 = (F3 - F3_min) * 255/(F3_max - F3_min)
            
        cv2.imshow("ifft_complex",F1_4.astype(uint8))
        cv2.imshow("ifft_single",F3.astype(uint8))
        cv2.imshow("mouse",F2)
            
            
    cv2.destroyAllWindows()            
    print("Finished")

C:\Users\mitu\Anaconda3_1\lib\site-packages\ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide


Finished
